In [1]:
import requests, json, env
from requests.auth import HTTPBasicAuth


def getBearerToken(env: env):
	if env.token != "": return env.token

	print("no token")

	req = requests.post(
		url=f"{env.url}/webapi/rest/auth",
		auth=HTTPBasicAuth(env.login, env.password),
		headers={
			"Content-Type": "application/json",
		},
		data={
			"client_id": env.login,
			"client_secret": env.password
		}
	)

	token: str = f"Bearer {json.loads(req.content)['access_token']}"

	with open("./env.py") as file: 
		data = file.read()
		data = data.replace(
			data.split('\n')[-1],
			f'token: str = "{token}"'
		)

	with open("./env.py", "w") as file:
		file.write(data)

	return token


In [2]:
def errorHandler(data = False):
	if data != False:
		print(data)
	print("Unexpected error")

In [3]:
import env
from typing import Literal


token = getBearerToken(env)

def getMondraker(offset: int = 0) -> list | Literal[False]:
	req = requests.get(
		url=f"{env.url}/webapi/rest/products",
		headers={
			"Authorization": token,
			"Content-Type": "application/json",
		},
		params={
			"filters": json.dumps({
				"producer_id": 251
			}),
			"limit": 50,
			"offset": offset
		}
	)

	if not req.ok:
		errorHandler(req)
		return False
	return json.loads(req.content)


In [4]:
from typing import Literal
import math


def getAllMondraker() -> list | Literal[False]:
	firstReq: list | Literal[False] = getMondraker()
	if firstReq == False: return False

	allReq: int = math.ceil(int(firstReq["count"]) / 50)
	data: list = firstReq["list"]

	for i in range(1, allReq):
		req: list | Literal[False] = getMondraker(i * 50)

		if req == False: return data
		data += req["list"]

	return data

In [5]:
import env
from typing import Literal


token: str = getBearerToken(env)

def getStocksByProductsList(mondraker) -> dict[str, list | Literal[False]]:
	def getStock(id: int) -> list | Literal[False]:
		req = requests.get(
			url=f"{env.url}/webapi/rest/product-stocks",
			headers={
				"Authorization": token,
				"Content-Type": "application/json",
			},
			params={
				"filters": json.dumps({
					"product_id": id
				})
			}
		)

		if not req.ok:
			errorHandler(req)
			return False
		return json.loads(req.content)

	stocks: dict[str, list | Literal[False]] = {}

	for bike in mondraker:
		try:
			stocks[bike["additional_producer"]] = getStock(bike["product_id"])
		except:
			pass

	return stocks

In [6]:
from typing import Literal
import requests, xmltodict


def getB2BAvailability() -> dict | Literal[False]:
	bikes = requests.get(
		url="https://www.blizzardski.cz/b2b_feeds/feedy_prodejci/blizzard_b2b_MONDRAKER.xml",
		headers={ "Content-Type": "application/xml" }
	)
	availability = requests.get(
		url="https://www.blizzardski.cz/b2b_feeds/availability_product.xml",
		headers={ "Content-Type": "application/xml" }
	)
	if not bikes.ok or not availability.ok:
		errorHandler()
		return False

	bikes = xmltodict.parse(bikes.content)["SHOP"]["SHOPITEM"]
	availability = xmltodict.parse(availability.content)["item_list"]["item"]


	def findAvailabilityByEan(ean: str) -> dict | Literal[False]:
		for available in availability:
			if available["ean"] == ean:
				return available
		return False

	fixed: dict = {}

	for bike in bikes:
		availableSizes: dict = {}

		if isinstance(bike["VARIANT"], dict):
			available = findAvailabilityByEan(bike["VARIANT"]["EAN"])
			if not available: continue

			size: str = bike["VARIANT"]["VAL"].split(" ")[-1]
			availableSizes[size] = {
				**available,
				**bike["VARIANT"]
			}

		else:
			for variant in bike["VARIANT"]:
				available = findAvailabilityByEan(variant["EAN"])
				if not available: continue

				size: str = variant["VAL"].split(" ")[-1]
				availableSizes[size] = {
					**available,
					**variant
				}


		if availableSizes == {}: continue
		fixed[bike["ITEMGROUP_ID"]] = {
			"PRODUCT": bike["PRODUCT"],
			"ITEMGROUP_ID": bike["ITEMGROUP_ID"],
			"PARAM": availableSizes
		}

	return fixed



In [7]:
mondraker = getAllMondraker()

In [8]:
stocks = getStocksByProductsList(mondraker)

In [9]:
b2bAvailability = getB2BAvailability()

In [25]:
import json


def updateStockAvailability(id: str, stock: str):
	req = requests.put(
		url=f"{env.url}/webapi/rest/product-stocks/{id}",
		headers={
			"Authorization": token,
			"Content-Type": "application/json",
		},
		data=json.dumps({
			"stock": stock
		})
	)

	print(f"code: {req.status_code} ok: {req.ok} content: {req.content}")





In [26]:
import json


def isVariant(code: str) -> bool:
	code = code.upper()
	allowedEnds = ["-S", "-M", "-L", "-XL"]

	for chars in allowedEnds:
		if code[-len(chars):] == chars: return True

	return False


bikeWithNoStock = 0

for key, stock in stocks.items():

	###############################
	# NA CZAS DEBUGOWANIA
	#
	# do updatowania tylko i wyłącznie jednego produktu
	# kod: MD23009
	# Rozmiar: L
	###############################
	if key != "010.23121": continue
	isAvailable = False


	print(" ")
	try:
		print(b2bAvailability[key]["PRODUCT"])
	except:
		print("brak nazwy")

	for singleSize in stock["list"]:
		if not isVariant(singleSize["code"]): continue

		sizeId: str = singleSize["code"].split("-")[-1].upper()

		try:
			availability = b2bAvailability[key]["PARAM"][sizeId]["stock_quantity"]
			print(f"Rozmiar: {sizeId} Dostępność: {singleSize["stock"]} Hurtownia: {availability}")
			updateStockAvailability(singleSize["stock_id"], availability)
		except:
			print(f"Brak dostępności dla rozmiaru: {sizeId}")


		isAvailable = True

	if not isAvailable: bikeWithNoStock += 1

print(bikeWithNoStock)






 
MONDRAKER Foxy Carbon XR MIND, racing silver/yellow
Brak dostępności dla rozmiaru: S
Brak dostępności dla rozmiaru: M
Rozmiar: L Dostępność: 0 Hurtownia: 1
code: 200 ok: True content: b'1'
Brak dostępności dla rozmiaru: XL
0


In [46]:
print(json.dumps(b2bAvailability, indent=2))

{
  "010.23005": {
    "PRODUCT": "MONDRAKER Chrono, desert grey/black",
    "ITEMGROUP_ID": "010.23005",
    "PARAM": {
      "M": {
        "@id": "27218",
        "ean": "8435392857239",
        "stock_quantity": "9",
        "delivery_date": "0",
        "stock_quantity_VOTesin": "9",
        "delivery_date_VOTesin": "0",
        "EAN": "8435392857239",
        "PRODUCTNO": "0000038140",
        "VAL": "Velikost M"
      },
      "XL": {
        "@id": "27973",
        "ean": "8435392857253",
        "stock_quantity": "10",
        "delivery_date": "0",
        "stock_quantity_VOTesin": "10",
        "delivery_date_VOTesin": "0",
        "EAN": "8435392857253",
        "PRODUCTNO": "0000038142",
        "VAL": "Velikost XL"
      },
      "L": {
        "@id": "27219",
        "ean": "8435392857246",
        "stock_quantity": "10",
        "delivery_date": "0",
        "stock_quantity_VOTesin": "10",
        "delivery_date_VOTesin": "0",
        "EAN": "8435392857246",
        "PRO